In [1]:
!pip install google-generativeai==0.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 28.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.6
    Uninstalling google-ai-generativelanguage-0.6.6:
      Successfully uninstalled google-ai-generativelanguage-0.6.6
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.7.2
    Uninstalling google-generativeai-0.7.2:
      Successfully uninstalled google-generativeai-0.7.2


In [2]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key= userdata.get('GOOGLE_API_KEY'))

In [3]:
# @title 함수 호출 기초
import google.generativeai as genai

def get_price(product: str)-> int:
    """제품의 가격을 알려주는 함수

    Args:
        theme: 제품명
    """
    return 1000


def get_temperature(city: str)-> float:
    """도시의 온도를 알려주는 함수

    Args:
        genre: 도시명
    """
    return 20.5

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=[get_price, get_temperature])
chat_session = model.start_chat(enable_automatic_function_calling=True)
response = chat_session.send_message("서울의 온도는?")
print(response.text)


서울의 온도는 20.5도입니다. 



In [4]:
# @title 함수 호출 구현하기
prod_database = {
    "갤럭시 S24": {"재고": 10, "가격": 1_700_000},
    "갤럭시 S23": {"재고": 5, "가격": 1_300_000},
    "갤럭시 S22": {"재고": 3, "가격": 1_100_000},
}

In [5]:
def is_product_available(product_name: str)-> bool:
    """특정 제품의 재고가 있는지 확인한다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        if prod_database[product_name]["재고"] > 0:
            return True
    return False

def get_product_price(product_name: str)-> int:
    """제품의 가격을 가져온다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        return prod_database[product_name]["가격"]
    return None

def place_order(product_name: str, address: str)-> str:
    """제품 주문결과를 반환한다.
    Args:
        product_name: 제품명
        address: 배송지
    """
    if is_product_available(product_name):
        prod_database[product_name]["재고"] -= 1
        return "주문 완료"
    else:
        return "재고 부족으로 주문 불가"



In [6]:
function_repoistory = {
    "is_product_available": is_product_available,
    "get_product_price": get_product_price,
    "place_order": place_order
}


In [7]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=function_repoistory.values()
)

In [8]:
def correct_response(response):
    part = response.candidates[0].content.parts[0]
    if part.function_call:
        for k, v in part.function_call.args.items():
            byte_v = bytes(v, "utf-8").decode("unicode_escape")
            corrected_v = bytes(byte_v, "latin1").decode("utf-8")
            part.function_call.args.update({k:  corrected_v})

In [9]:
chat_session = model.start_chat()
prompt = "갤럭시 S24 판매 중인가요?"
response = chat_session.send_message(prompt)
correct_response(response)
print(response.candidates[0].content)


parts {
  function_call {
    name: "is_product_available"
    args {
      fields {
        key: "product_name"
        value {
          string_value: "\352\260\244\353\237\255\354\213\234 S24"
        }
      }
    }
  }
}
role: "model"



In [10]:
part = response.candidates[0].content.parts[0]
if part.function_call:
    function_call =  part.function_call
    function_name = function_call.name
    function_args = {k: v for k, v in function_call.args.items()}
    print(f"{function_name} args=>: {function_args}")
    function_result = function_repoistory[function_name](**function_args)
    print(f"{function_name} result=>: {function_result}")


is_product_available args=>: {'product_name': '갤럭시 S24'}
is_product_available result=>: True


In [11]:
import google.ai.generativelanguage as glm

part = glm.Part(
    function_response=glm.FunctionResponse(
        name="is_product_available",
        response={
            "content": function_result,
        },
    )
)
print(part)
response = chat_session.send_message(part)
correct_response(response)
print("-"*50)
print(response.candidates[0].content)


function_response {
  name: "is_product_available"
  response {
    fields {
      key: "content"
      value {
        bool_value: true
      }
    }
  }
}

--------------------------------------------------
parts {
  text: "\353\204\244, \355\214\220\353\247\244 \354\244\221\354\236\205\353\213\210\353\213\244. \360\237\230\212 \n"
}
role: "model"



In [12]:
# @title 스마트폰 주문 챗봇 구현
prod_database = {
    "갤럭시 S24": {"재고": 10, "가격": 1_700_000},
    "갤럭시 S23": {"재고": 5, "가격": 1_300_000},
    "갤럭시 S22": {"재고": 3, "가격": 1_100_000},
}

In [13]:
def is_product_available(product_name: str)-> bool:
    """특정 제품의 재고가 있는지 확인한다.

    Args:
        product_name: 제품명
    """

    if product_name in prod_database:
        if prod_database[product_name]["재고"] > 0:
            return True
    return False

def get_product_price(product_name: str)-> int:
    """제품의 가격을 가져온다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        return prod_database[product_name]["가격"]
    return None

def place_order(product_name: str, address: str)-> str:
    """제품 주문결과를 반환한다.
    Args:
        product_name: 제품명
        address: 배송지
    """
    if is_product_available(product_name):
        prod_database[product_name]["재고"] -= 1
        return "주문 완료"
    else:
        return "재고 부족으로 주문 불가"

function_repoistory = {
    "is_product_available": is_product_available,
    "get_product_price": get_product_price,
    "place_order": place_order
}

def correct_response(response):
    part = response.candidates[0].content.parts[0]
    if part.function_call:
        for k, v in part.function_call.args.items():
            byte_v = bytes(v, "utf-8").decode("unicode_escape")
            corrected_v = bytes(byte_v, "latin1").decode("utf-8")
            part.function_call.args.update({k:  corrected_v})


In [14]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=function_repoistory.values()
)

In [15]:
chat_session = model.start_chat(history=[])
queries = ["갤럭시 S24 판매 중인가요?", "가격은 어떻게 되나요?", "서울시 종로구 종로1가 1번지로 배송부탁드립니다"]

for query in queries:
    print(f"\n사용자: {query}")
    response = chat_session.send_message(query)
    correct_response(response)
    part = response.candidates[0].content.parts[0]

    if part.function_call:
        function_call =  part.function_call
        function_name = function_call.name
        function_args = {k: v for k, v in function_call.args.items()}
        function_result = function_repoistory[function_name](**function_args)
        part = glm.Part(
                function_response=glm.FunctionResponse(
                    name=function_name,
                    response={
                        "content": function_result,
                    },
                )
            )
        response = chat_session.send_message(part)
        print(f"모델: {response.candidates[0].content.parts[0].text}")


사용자: 갤럭시 S24 판매 중인가요?
모델: 네, 판매 중입니다. 😊  
혹시 궁금한 점이 있으신가요? 


사용자: 가격은 어떻게 되나요?
모델: 갤럭시 S24 가격은 1,700,000원 입니다. 


사용자: 서울시 종로구 종로1가 1번지로 배송부탁드립니다
모델: 주문 완료되었습니다!  
서울시 종로구 종로1가 1번지로 배송될 예정이며,  
배송 관련 문의는 고객센터로 연락 부탁드립니다. 
 
 감사합니다. 😊 

